文本预处理-文本张量的表示方法
===

将一段文本使用张量进行表示，其中一般将词汇表示成向量，称作词向量，再由各个词向量按顺序组成矩阵形成文本表示

# 1.One-Hot编码
独热编码，将每个词表示成具有n个元素的向量，这个词向量中只有一个元素是1，其他元素都是0，不同词汇元素为0的位置不同，其中n的大小是整个语料中不同词汇的总数

In [17]:
from sklearn.externals import joblib
from keras.preprocessing.text import Tokenizer

vocab = ("周杰伦", "陈奕迅", "王力宏", "李宗盛", "吴亦凡", "鹿晗")
t = Tokenizer(num_words=None, char_level=False)
t.fit_on_texts(vocab)

for token in vocab:
    zero_list = [0] * len(vocab)
    token_index = t.texts_to_sequences([token])[0][0] - 1
    zero_list[token_index] = 1
    print(token, "的One-Hot编码为:", zero_list)

tokenizer_path = "./Tokenizer"
joblib.dump(t, tokenizer_path)

周杰伦 的One-Hot编码为: [1, 0, 0, 0, 0, 0]
陈奕迅 的One-Hot编码为: [0, 1, 0, 0, 0, 0]
王力宏 的One-Hot编码为: [0, 0, 1, 0, 0, 0]
李宗盛 的One-Hot编码为: [0, 0, 0, 1, 0, 0]
吴亦凡 的One-Hot编码为: [0, 0, 0, 0, 1, 0]
鹿晗 的One-Hot编码为: [0, 0, 0, 0, 0, 1]


['./Tokenizer']

In [20]:
from sklearn.externals import joblib
t = joblib.load(tokenizer_path)

token = "李宗盛"
token_index = t.texts_to_sequences([token])[0][0] - 1
zero_list = [0] * len(vocab)
zero_list[token_index] = 1
print(token, "的One-Hot编码为:", zero_list)

李宗盛 的One-Hot编码为: [0, 0, 0, 1, 0, 0]


优势：操作简单，容易理解

劣势：完全割裂了词与词之间的关联，而且在大语料集下，每个向量的长度过大，占据大量内存，所以现在用这种办法很少了。

# 2.Word2Vec
这是一种流行的将词汇表示成词向量的无监督训练方法，该过程将构建神经网络模型，将网络参数作为词汇的词向量表示，它包含CBOW和skipgram两种训练方式

## 2.1.CBOW(Continuous bag of words，连续词袋模式)模式
给定一段用于训练的文本语料，在选定某段长度(窗口)作为研究对象，使用上下词汇预测目标词汇

![Image](images/003.png)

图中(the quick brown fox jumps over the lazy dog)窗口大小为9，使用前后4个词汇对目标词汇进行预测。相当于我知道了上文的四个单词，也知道了下文的四个单词，来预测中间的Jumps

假设我们给定的训练预料只有一句话:Hope can set you free，窗口大小为3，因此模型的第一个训练样本来自Hope can set，因为是CBOW模式，所以将使用Hope和Set作为输入，Can作为输出，在模型训练时，Hope、can、set等词汇都是用他们的one-hot编码，如图所示：每个one-hot编码的单词与各自的变换矩阵(即参数矩阵$3 \times 5$这里的3指的是得到的词向量维度)相城之后再相加，得到上下文表示矩阵$3 \times 1$

![images](images/004.png)

接着，将上下文表示矩阵与变换矩阵(参数矩阵$5 \times 3$，所有的变换矩阵共享参数)相乘，得到$5 \times 1$进行损失的计算，然后更新网络参数完成一次模型迭代。

![images](images/005.png)

最后窗口按序向后移动，重新更新参数，知道所有语料都被遍历完成，得到最终的变换矩阵$3 \times 5$，这个变换矩阵与每个词汇的one-hot编码$5 \times 1$相乘，得到的$3 \times 1$的矩阵就是该词汇的word2vec张量表示

## 2.2.Skip-Gram过程
Skip-Gram与CBOW的过程正好相反，它是给定中间词，去预测上下文的别的词。

假设我们给定的训练语料只有一句话:Hope can set you free，窗口大小为3，因此模型的第一个训练样本来自Hope can set，那么输入就是can，而输出就是hope与set。在模型训练时，Hope，can，set等词汇都是用它们的one-hot编码，如图所示，将can的one-hot编码与变换矩阵(即参数矩阵$3 \times 5$，这里的3是指最后得到的词向量的维度)相乘，得到目标词汇表示矩阵$3 \times 1$

接着，将目标词汇表示矩阵与多个变换矩阵$5 \times 3$相乘，得到多个$5 \times 1$的结果矩阵，它将与我们hope和set对应的one-hot编码矩阵进行损失的计算，然后更新网络参数完成一次模型迭代

![images](images/006.png)

最后窗口按序向后移动，重新更新参数，知道所有语料遍历完成，得到最终的变换矩阵即参数矩阵$3 \times 5$，这个变换矩阵与每个词汇的one-hot编码相乘，得到的$3 \times 1$的矩阵就是该词汇的word2vec张量表示

# 3.Word Embedding